
Question 1.
I have added some dummy data to validate the queries.

This <span style="color:red">word</span> is not black.
Text <span style="font-family: Arial; font-size:1.4em;color:gold;"> Golden </p> Text

In [458]:
#!pip install pandasql
import pandas as pd
import numpy as np
import pandasql as ps
question1_df = pd.read_csv("C:/Users/tc302599/Documents/Faceit/Q1.csv")

In [459]:
question1_df.head()

,user_id,match_id,game,created_at,membership,faction,winner
0,ab542a,e21887,csgo,02/01/2018 18:45,free,faction1,faction1
1,ef72da,df891f,dota2,02/01/2018 08:20,free,faction2,faction1
2,f5c776,1p5c47,wot_RU,30/12/2017 15:25,free,faction1,faction2
3,5a278a,af14e8,csgo,01/01/2018 14:27,premium,faction2,faction2
4,ae346d,af14e8,csgo,01/01/2018 14:27,free,faction2,faction2


In [460]:
list(question1_df.columns.values)

['user_id',
 'match_id',
 'game',
 'created_at',
 'membership',
 'faction',
 'winner']

In [461]:
question1_df.groupby(['winner']).count()

,user_id,match_id,game,created_at,membership,faction
winner,,,,,,
faction1,53,53,53,53,53,53
faction2,13,13,13,13,13,13


In [462]:
#Validate we are getting unique counts for each match, and only pulling matches with premium users 
#and are correctly extracting the year from created_at 
#q1 = """SELECT match_id,
#              substr(created_at,7,4) as year,
#              membership,
#              count(distinct match_id) as Number_Of_Matches
#       FROM question1_df
#       where membership='premium' and substr(created_at,7,4)='2018'
#       group by match_id, membership
#       """

#Finalise Query
q1 = """SELECT substr(created_at,7,4) as year,
               count(distinct match_id) as Number_Of_Matches
        FROM question1_df
        where membership='premium' and substr(created_at,7,4)='2018'
        """

print(ps.sqldf(q1, locals()))

   year  Number_Of_Matches
0  2018                  3


In [463]:
q3 = """
    SELECT   user_id, game, MAX(seq_length) AS longest_sequence
    FROM (
          SELECT   
            user_id, 
            game,
            COUNT(*) AS seq_length
          FROM ( SELECT *, 
                   SUM(new_group) OVER(ORDER BY user_id, game) AS group_no
                 FROM ( SELECT   *,
                                 LAG(win_flag) OVER(ORDER BY user_id, created_at) AS new_group
                                 FROM (select *, 
                                 case when faction=winner then 1 else 0 end as win_flag 
                        FROM question1_df
                        )))
          WHERE win_flag = 1
          GROUP BY user_id, group_no)
   GROUP BY user_id, game
   ORDER BY 3 DESC, 3, 1
              
        """

print(ps.sqldf(q3, locals()))

    user_id    game  longest_sequence
0    ace797   csgo2                 9
1   aaaaaaa    test                 7
2   aaaaaaa  csgo20                 6
3    ab542a    csgo                 6
4    5a278a    csgo                 3
5    ae346d    csgo                 3
6    ace797  csgo10                 1
7    ace797  csgo11                 1
8    ace797  csgo12                 1
9    ace797  csgo13                 1
10   ace797  csgo14                 1
11   ace797  csgo15                 1
12   ace797  csgo16                 1
13   ace797  csgo17                 1
14   ace797  csgo18                 1
15   ace797  csgo19                 1
16   ace797  csgo20                 1
17   ace797   csgo3                 1
18   ace797   csgo4                 1
19   ace797   csgo5                 1
20   ace797   csgo6                 1
21   ace797   csgo7                 1
22   ace797   csgo8                 1
23   ace797   csgo9                 1


In [464]:
q2 = """
    SELECT   user_id, game, MAX(seq_length) AS longest_sequence
    FROM (
          SELECT   
            user_id, 
            game,
            COUNT(*) AS seq_length
          FROM ( SELECT *, 
                   SUM(new_group) OVER(ORDER BY user_id, game) AS group_no
                 FROM ( SELECT   *,
                                 LAG(win_flag) OVER(ORDER BY user_id, created_datetime) AS new_group
                                 FROM (select *, 
                                 case when faction=winner then 1 else 0 end as win_flag,
                                 DATETIME(substr(created_at,7,4)||'-'||substr(created_at,4,2)||'-'||substr(created_at,1,2)
                                          ||substr(created_at,11,6) ) as created_datetime     
                        FROM question1_df
                        )))
          WHERE win_flag = 1
          GROUP BY user_id, group_no)
   GROUP BY user_id, game
   ORDER BY 3 DESC, 3, 1
              
        """

print(ps.sqldf(q2, locals()))

    user_id    game  longest_sequence
0    ace797   csgo2                 9
1   aaaaaaa    test                 7
2   aaaaaaa  csgo20                 6
3    ab542a    csgo                 6
4    5a278a    csgo                 3
5    ae346d    csgo                 3
6    ace797  csgo10                 1
7    ace797  csgo11                 1
8    ace797  csgo12                 1
9    ace797  csgo13                 1
10   ace797  csgo14                 1
11   ace797  csgo15                 1
12   ace797  csgo16                 1
13   ace797  csgo17                 1
14   ace797  csgo18                 1
15   ace797  csgo19                 1
16   ace797  csgo20                 1
17   ace797   csgo3                 1
18   ace797   csgo4                 1
19   ace797   csgo5                 1
20   ace797   csgo6                 1
21   ace797   csgo7                 1
22   ace797   csgo8                 1
23   ace797   csgo9                 1


In [465]:
q2 = """
    SELECT   user_id, game, MAX(seq_length) AS longest_sequence
    FROM (
          SELECT   
            user_id, 
            game,
            COUNT(*) AS seq_length
          FROM ( SELECT *, 
                   SUM(new_group) OVER(ORDER BY user_id, game, created_datetime) AS group_no
                 FROM ( SELECT   *,
                                 case when LAG(win_flag) OVER(ORDER BY user_id, created_datetime)=win_flag then 0 else 1 end AS new_group
                                 FROM (select *, 
                                 case when faction=winner then 1 else 0 end as win_flag,
                                 DATETIME(substr(created_at,7,4)||'-'||substr(created_at,4,2)||'-'||substr(created_at,1,2)
                                          ||substr(created_at,11,6) ) as created_datetime     
                        FROM question1_df
                        )))
          WHERE win_flag = 1
          GROUP BY user_id, group_no)
   GROUP BY user_id, game
   ORDER BY 3 DESC, 3, 1
              
        """

print(ps.sqldf(q2, locals()))


   user_id    game  longest_sequence
0   ace797   csgo2                11
1   ace797  csgo10                10
2  aaaaaaa  csgo20                 7
3  aaaaaaa    test                 6
4   ab542a    csgo                 6
5   5a278a    csgo                 3
6   ae346d    csgo                 3


In [466]:
from scipy import stats
def two_proprotions_confint(success_a, size_a, success_b, size_b, significance = 0.05):
    """
    A/B test for two proportions;
    given a success a trial size of group A and B compute
    its confidence interval;
    resulting confidence interval matches R's prop.test function

    Parameters
    ----------
    success_a, success_b : int
        Number of successes in each group

    size_a, size_b : int
        Size, or number of observations in each group

    significance : float, default 0.05
        Often denoted as alpha. Governs the chance of a false positive.
        A significance level of 0.05 means that there is a 5% chance of
        a false positive. In other words, our confidence level is
        1 - 0.05 = 0.95

    Returns
    -------
    prop_diff : float
        Difference between the two proportion

    confint : 1d ndarray
        Confidence interval of the two proportion test
    """
    prop_a = success_a / size_a
    prop_b = success_b / size_b
    var = prop_a * (1 - prop_a) / size_a + prop_b * (1 - prop_b) / size_b
    se = np.sqrt(var)

    # z critical value
    confidence = 1 - significance
    z = stats.norm(loc = 0, scale = 1).ppf(confidence + significance / 2)

    # standard formula for the confidence interval
    # point-estimtate +- z * standard-error
    prop_diff = prop_b - prop_a
    confint = prop_diff + np.array([-1, 1]) * z * se
    return prop_diff, confint

In [467]:
question2_df = pd.read_csv("C:/Users/tc302599/Documents/Faceit/client__1_.csv")

In [468]:
question2_df.head()

,user_id,first_day,latest_day,lifespan,client,membership_type,joined_at_least_1_hub
0,594132bd-7b3e-4ec4-b752-06ee1711d297,11/10/2017,11/10/2017,0,new_client,free,NaN
1,533f08c8-e1ee-4925-96d9-9aeeba55b5ba,11/10/2017,11/10/2017,0,new_client,free,NaN
2,0840ad75-5337-4193-8cf8-0348b126e697,11/10/2017,11/10/2017,0,web_client,free,NaN
3,6e51c17f-4a81-4bb9-9f35-f0e2839da86e,11/10/2017,11/10/2017,0,new_client,free,1.0
4,f0dde747-b394-4a75-a74f-5e4adec18a28,11/10/2017,11/10/2017,0,new_client,free,NaN


In [469]:
question2_df.groupby(['client']).agg(['count'])

,user_id,first_day,latest_day,lifespan,membership_type,joined_at_least_1_hub
,count,count,count,count,count,count
client,,,,,,
new_client,1809,1809,1809,1809,1809,1557
web_client,1496,1496,1496,1496,1496,1253


In [470]:
#web_client is a - new_client is b 


size_a=question2_df.query("client == 'web_client'")['user_id'].count()
print(size_a)
size_b=question2_df.query("client == 'new_client'")['user_id'].count()
print(size_b)


success_a=question2_df.query("client == 'web_client'")['joined_at_least_1_hub'].sum()
print(success_a)
success_b=question2_df.query("client == 'new_client'")['joined_at_least_1_hub'].sum()
print(success_b)

1496
1809
1253.0
1557.0


In [471]:
prop_b=question2_df.query("client == 'new_client'")['joined_at_least_1_hub'].sum()/size_b
print(prop_b)
prop_a=question2_df.query("client == 'web_client'")['joined_at_least_1_hub'].sum()/size_a
print(prop_a)
print(prop_b-prop_a)

0.8606965174129353
0.8375668449197861
0.023129672493149234


In [472]:
two_proprotions_confint(success_a, size_a, success_b, size_b, significance = 0.1)

(0.023129672493149234, array([0.00250527, 0.04375408]))

In [473]:
two_proprotions_confint(success_a, size_a, success_b, size_b, significance = 0.05)

(0.023129672493149234, array([-0.00144582,  0.04770517]))

In [474]:
question3_df = pd.read_csv("C:/Users/tc302599/Documents/Faceit/toxicity_xls.csv")

In [475]:
question3_df.head()

,flirtation,identity_attack,insult,severe_toxicity,sexually_explicit,threat,label
0,0.593828,0.563516,0.849090,0.864632,0.777347,0.602494,Offensive
1,0.213193,0.407253,0.925010,0.856451,0.456983,0.592931,Offensive
2,0.474532,0.323574,0.710831,0.747318,0.933715,0.208848,Very offensive
3,0.503426,0.407557,0.796685,0.854638,0.955973,0.343336,Neutral
4,0.394807,0.170078,0.561849,0.766563,0.459300,0.223698,Profanity


In [476]:
#Extremely offensive, Hate speech, Very offensive
#Neutral, Offensive, Profanity, Unknown 


def toxic_flag(row):
  if row['label'] in('Very offensive', 'Profanity','Extremely offensive', 'Hate speech'): 
    return 1
  else:
    return 0

question3_df['toxic_flag']=question3_df.apply(toxic_flag, axis=1)

In [477]:
question3_df.groupby(['label']).agg(['sum'])

,flirtation,identity_attack,insult,severe_toxicity,sexually_explicit,threat,toxic_flag
,sum,sum,sum,sum,sum,sum,sum
label,,,,,,,
Extremely offensive,180.364857,393.906974,387.335691,375.177752,232.836210,192.049122,426
Hate speech,36.682110,80.831353,78.129105,77.723241,49.936092,46.011031,86
Neutral,308.974074,297.055393,513.944161,607.201097,266.745337,574.404952,0
Offensive,2525.994241,2709.206502,5104.892953,5028.814834,3508.575642,2259.559008,0
Profanity,1431.329736,900.550534,2450.857810,2678.553772,1878.485324,1075.869009,3430
Unknown,61.788665,64.197408,72.516177,78.163724,60.977095,56.558707,0
Very offensive,456.617007,812.848039,1011.287835,1016.964380,681.328250,658.691246,1138


In [478]:
question3_df.groupby(['toxic_flag']).agg(['count'])

from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
target=lb.fit(question3_df['toxic_flag'])

In [479]:
#split training and test set
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(question3_df, test_size=0.2, random_state=42)
print(len(train_set))
print(len(test_set))

9600
2400


In [480]:
X_train = train_set.drop(['toxic_flag','label'], axis=1)
X_train.head()

,flirtation,identity_attack,insult,severe_toxicity,sexually_explicit,threat
9182,0.713375,0.184467,0.628916,0.722214,0.955973,0.209654
11091,0.497507,0.934033,0.919052,0.945149,0.868834,0.288530
6428,0.253752,0.715580,0.909873,0.737679,0.171833,0.270194
288,0.522886,0.606927,0.932669,0.886782,0.924914,0.398544
2626,0.474379,0.125789,0.727237,0.779466,0.902304,0.367039


In [481]:
Y_train = train_set['toxic_flag']
y_train=lb.transform(Y_train)
y_train=y_train.ravel()

X_test = test_set.drop(['toxic_flag','label'], axis=1)
Y_test = test_set['toxic_flag']
y_test=lb.transform(Y_test)
y_test=y_test.ravel()

In [482]:
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(max_iter=5, random_state=42, tol=-np.infty)
sgd_clf.fit(X_train, y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=5,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=42, shuffle=True, tol=-inf,
       validation_fraction=0.1, verbose=0, warm_start=False)

In [483]:
from sklearn.model_selection import cross_val_score
cross_val_score(sgd_clf, X_train, y_train, cv=3, scoring="accuracy")

array([0.456875, 0.619375, 0.598125])

In [484]:
from sklearn.model_selection import cross_val_predict
y_train_pred = cross_val_predict(sgd_clf, X_train, y_train, cv=3)

from sklearn.metrics import confusion_matrix
confusion_matrix(y_train, y_train_pred)

array([[3514, 2018],
       [2224, 1844]], dtype=int64)

In [485]:
from sklearn.metrics import precision_score, recall_score
precision_score(y_train, y_train_pred)

0.47747281201450026

In [486]:
y_test_pred = sgd_clf.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_test_pred)

0.5929166666666666

In [489]:
from sklearn.metrics import f1_score
f1_score(y_test, y_test_pred, average='weighted')

0.48451855872333494

In [490]:
from sklearn.ensemble import RandomForestClassifier

forest_clf = RandomForestClassifier(random_state=42, n_estimators=100)
forest_scores = cross_val_score(forest_clf, X_train, y_train, cv=10)
forest_scores.mean()

0.7209340168102758

In [491]:
forest_clf.fit(X_train, y_train)
y_train_pred = forest_clf.predict(X_train)
confusion_matrix(y_train, y_train_pred)

array([[5516,   16],
       [  20, 4048]], dtype=int64)

In [492]:
forest_clf.fit(X_train, y_train)
y_train_pred = forest_clf.predict(X_train)
confusion_matrix(y_train, y_train_pred)

array([[5516,   16],
       [  20, 4048]], dtype=int64)

In [493]:
y_test_pred = forest_clf.predict(X_test)
confusion_matrix(y_test, y_test_pred)


array([[1141,  247],
       [ 389,  623]], dtype=int64)

In [494]:
accuracy_score(y_test, y_test_pred)

0.735

In [495]:
f1_score(y_test, y_test_pred, average='weighted')

0.7314505662752137

In [435]:
import h2o
h2o.init()
from h2o.automl import H2OAutoML

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM 18.9 (build 11.0.3+12-LTS, mixed mode)
  Starting server from C:\Users\tc302599\AppData\Local\Continuum\anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\tc302599\AppData\Local\Temp\tmpklcksbek
  JVM stdout: C:\Users\tc302599\AppData\Local\Temp\tmpklcksbek\h2o_tc302599_started_from_python.out
  JVM stderr: C:\Users\tc302599\AppData\Local\Temp\tmpklcksbek\h2o_tc302599_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,Europe/London
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.8
H2O cluster version age:,6 months and 26 days !!!
H2O cluster name:,H2O_from_python_tc302599_tmsw8k
H2O cluster total nodes:,1
H2O cluster free memory:,3.971 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [450]:
train_temp=train_set.drop(['label'], axis=1)
train = h2o.H2OFrame(train_temp)
test_temp=test_set.drop(['label'], axis=1)
test = h2o.H2OFrame(test_temp)

x = train.columns
y = "toxic_flag"
x.remove(y)
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [451]:
aml = H2OAutoML(max_models = 30, max_runtime_secs=300, seed = 1)
aml.train(x = x_cols, y = y , training_frame = train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [452]:
lb = aml.leaderboard
lb.head()
lb.head(rows=lb.nrows) # Entire leaderboard

model_id,auc,logloss,mean_per_class_error,rmse,mse
StackedEnsemble_BestOfFamily_AutoML_20200514_130448,0.789116,0.545484,0.290218,0.42758,0.182825
StackedEnsemble_AllModels_AutoML_20200514_130448,0.789032,0.545932,0.292533,0.427803,0.183015
GBM_5_AutoML_20200514_130448,0.782983,0.550671,0.304753,0.430545,0.185369
GBM_1_AutoML_20200514_130448,0.782643,0.551317,0.286475,0.430569,0.18539
GBM_2_AutoML_20200514_130448,0.782338,0.551327,0.297956,0.430761,0.185555
GBM_3_AutoML_20200514_130448,0.7815,0.551809,0.311124,0.431042,0.185798
GBM_grid_1_AutoML_20200514_130448_model_7,0.779944,0.553859,0.301448,0.431844,0.186489
DRF_1_AutoML_20200514_130448,0.776034,0.578139,0.298024,0.434485,0.188777
GBM_4_AutoML_20200514_130448,0.77513,0.559875,0.301148,0.434364,0.188672
XRT_1_AutoML_20200514_130448,0.773525,0.58157,0.315332,0.435515,0.189674


In [453]:
perf = aml.leader.model_performance(test)
perf


ModelMetricsBinomialGLM: stackedensemble
** Reported on test data. **

MSE: 0.1781269315093314
RMSE: 0.4220508636519198
LogLoss: 0.5354968204215205
Null degrees of freedom: 2399
Residual degrees of freedom: 2395
Null deviance: 3267.9991081093194
Residual deviance: 2570.3847380232983
AIC: 2580.3847380232983
AUC: 0.7966256506931234
pr_auc: 0.754532418293023
Gini: 0.5932513013862468

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3189035939733982: 


,,0,1,Error,Rate
0,0,915.0,473.0,0.3408,(473.0/1388.0)
1,1,218.0,794.0,0.2154,(218.0/1012.0)
2,Total,1133.0,1267.0,0.2879,(691.0/2400.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.318904,0.696797,255.0
1,max f2,0.163380,0.798095,346.0
2,max f0point5,0.560957,0.714468,147.0
3,max accuracy,0.560957,0.742083,147.0
4,max precision,0.914323,1.000000,0.0
5,max recall,0.084730,1.000000,399.0
6,max specificity,0.914323,1.000000,0.0
7,max absolute_mcc,0.560957,0.465448,147.0
8,max min_per_class_accuracy,0.379226,0.719741,230.0
9,max mean_per_class_accuracy,0.483186,0.725623,180.0



Gains/Lift Table: Avg response rate: 42.17 %, avg score: 40.92 %


,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,0.01,0.885749,2.371542,2.371542,1.000000,0.895349,1.000000,0.895349,0.023715,0.023715,137.154150,137.154150
1,,2,0.02,0.872486,2.272727,2.322134,0.958333,0.878942,0.979167,0.887146,0.022727,0.046443,127.272727,132.213439
2,,3,0.03,0.863599,1.877470,2.173913,0.791667,0.867874,0.916667,0.880722,0.018775,0.065217,87.747036,117.391304
3,,4,0.04,0.851598,2.173913,2.173913,0.916667,0.857344,0.916667,0.874878,0.021739,0.086957,117.391304,117.391304
4,,5,0.05,0.843218,2.075099,2.154150,0.875000,0.847533,0.908333,0.869409,0.020751,0.107708,107.509881,115.415020
5,,6,0.10,0.799032,2.114625,2.134387,0.891667,0.820267,0.900000,0.844838,0.105731,0.213439,111.462451,113.438735
6,,7,0.15,0.745105,1.936759,2.068511,0.816667,0.773354,0.872222,0.821010,0.096838,0.310277,93.675889,106.851120
7,,8,0.20,0.682337,1.719368,1.981225,0.725000,0.716272,0.835417,0.794825,0.085968,0.396245,71.936759,98.122530
8,,9,0.30,0.563281,1.521739,1.828063,0.641667,0.618367,0.770833,0.736006,0.152174,0.548419,52.173913,82.806324
9,,10,0.40,0.440637,1.067194,1.637846,0.450000,0.498680,0.690625,0.676675,0.106719,0.655138,6.719368,63.784585


In [457]:
h2o.cluster().shutdown()

H2O session _sid_8c7f closed.
